# **DIVE INTO CODE COURSE**
## **Sprint Deep Learning - Convolution Neural Network 2D**
**Student Name**: Doan Anh Tien<br>
**Student ID**: 1852789<br>
**Email**: tien.doan.g0pr0@hcmut.edu.vn

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import seaborn as sns
import random
from math import log2
from sklearn.metrics import accuracy_score

plt.style.use('ggplot')

### **[Problem 1] Creating a 2-D convolutional layer**

### **Support class and functions**

In [2]:
class GetMiniBatch:

    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
#         self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        self._stop = int(np.ceil(X.shape[0]/self.batch_size))
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [3]:
def cross_entropy_error(y, Z3):

    DELTA = 1e-7
    batch_size = y.shape[0]
    return -np.sum(y * np.log(Z3 + DELTA))/batch_size

class SimpleInitializer:

    def __init__(self, sigma):
        pass

    def Wx(self, n_features, n_nodes):
        w = np.zeros((n_features, n_nodes))
        return w

    def Wh(self, n_nodes):
        w = np.zeros((n_nodes, n_nodes))
        return w

    def B(self, n_nodes):
        b = np.zeros(n_nodes)
        return b

class Tanh:

    def forward(self, A):
        self.A = A
        return np.tanh(A)

    def backward(self, dZ):
        return dZ * (1 - (np.tanh(self.A))**2)


class Softmax:

    def __init__(self):
        self.loss = None

    def forward(self, X):

        X = X.T
        y = np.exp(X) / np.sum(np.exp(X), axis=0)

        return y.T

    def backward(self, Z3, y):
        batch_size = y.shape[0]
        ret = (Z3 - y)/batch_size

        self.loss = cross_entropy_error(y, Z3)

        return ret


class ReLU:

    def __init__(self):
        self.x = None

    def forward(self, X):

        self.x = X

        return np.maximum(0, X)

    def backward(self, X):

        return np.where(self.x > 0, X, 0)


class XavierInitializer:

    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):

        sigma = 1.0 / np.sqrt(n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):

        b = np.random.randn(n_nodes2)
        return b

class HeInitializer:

    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):

        sigma = np.sqrt( 2.0 / n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):

        b = np.random.randn(n_nodes2)
        return b    
    
    
class AdaGrad:

    def __init__(self, lr):
        self.lr = lr
        self.HW = 1
        self.HB = 1

    def update(self, layer):
        self.HW += layer.dW**2
        self.HB += layer.dB**2
        layer.W -= self.lr * np.sqrt(1/self.HW) * layer.dW
        layer.B -= self.lr * np.sqrt(1/self.HB) * layer.dB
    
class SGD:

    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return

**Fully Connected Class**

In [28]:
class SimpleRNN:
    """
    Number of nodes Fully connected layer from n_nodes1 to n_nodes2
    Parameters
    ----------
    n_nodes1 : int
      Number of nodes in the previous layer
    n_nodes2 : int
      Number of nodes in the later layer
    initializer: object
      Instance of initialization method
    optimizer: object
      Instance of optimization method
    """
    def __init__(self, n_features, n_nodes, initializer, optimizer):
        self.optimizer = optimizer
        self.initializer = initializer

        self.n_features = n_features
        self.n_nodes = n_nodes

        self.Wx = self.initializer.Wx(self.n_features, self.n_nodes)
        self.Wh = self.initializer.Wh(self.n_nodes)

        self.B = self.initializer.B(self.n_nodes)

        # Number of states
        self.h = None
        self.x = None

        self.dWx = None
        self.dWb = None
        self.dB = None

    def forward(self, X):
        
        Wx = self.Wx
        Wh = self.Wh
        B = self.B

        # The input x to the entire RNN will be passed in an array like (batch_size, n_sequences, n_features)
        batch_size, n_sequences, n_features = X.shape
        n_features, n_nodes = Wx.shape

        h = np.empty((batch_size, n_sequences, n_nodes), dtype='f')

        for seq in range(n_sequences):
          x = X[:, seq, :]
          h_prev = self.h

          a = np.dot(x, Wx) + np.dot(h_prev, Wh) + B
          h_next = np.tanh(a)

          self.x = x
          self.h = h_next
          h[:, seq, :] = self.h

        self.state = h
        
        return self.h

    def backward(self, dA):

        self = self.optimizer.update(self)

        return None

In [29]:
class ScratchSimpleRNNClassifier():
    """
    Convolution neural network classifier with configurable structure
    Parameters
    ----------
    Attributes
    ----------
    """
    def __init__(self, epoch, batch_size, n_features, n_nodes, lr, sigma=0.01, optimizer=SGD, activation=Tanh, initializer=SimpleInitializer, verbose=False):
        self.batch_size = batch_size
        self.epoch = epoch
        self.lr = lr     
        self.sigma = sigma
        self.verbose = verbose

        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation

        self.RNN1 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation1 = self.activation()
        self.RNN2 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation2 = self.activation()
        self.RNN3 = SimpleRNN(self.n_features, self.n_nodes, self.initializer(self.sigma), self.optimizer(self.lr))
        self.activation3 = Softmax()

        self.loss_list = []
        self.acc_list = []

    def fit(self, X, y, X_val=None, y_val=None):

        Z3 = self.forward(X)
        self.backward(X, y, Z3)

        loss = self.activation3.loss

        self.loss_list.append(loss)


    def forward(self, X):

        A1 = self.RNN1.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.RNN2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.RNN3.forward(Z2)
        Z3 = self.activation3.forward(A3)

        return Z3

    def backward(self, X, y, Z4):

        dA3 = self.activation3.backward(Z4, y)
        dZ2 = self.RNN3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.RNN2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dZ0 = self.RNN1.backward(dA1)

    def predict(self, X):

        y_pred = self.forward(_X)

        return y_pred.argmax(axis=1)

In [26]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [31]:
rnn = SimpleRNN(n_features=n_features, n_nodes=n_nodes, initializer=SimpleInitializer(0.01), optimizer=SGD)
rnn.Wx = w_x
rnn.Wh = w_h
rnn.h = h
rnn.B = b

hs = rnn.forward(x)
print('Activation output value:', hs)
print('State:\n', rnn.state)

Activation output value: [[0.79494228 0.81839002 0.83939649 0.85584174]]
State:
 [[[0.76188797 0.76213956 0.762391   0.7625584 ]
  [0.792209   0.8141834  0.8340491  0.84977716]
  [0.79494226 0.81839    0.8393965  0.85584176]]]
